<a href="https://colab.research.google.com/github/tvani2/Walmart-Recruiting---Store-Sales-Forecasting/blob/main/Arima.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
print("wmae =", 2)

n_polynomials=5 → max_steps=2600 → h=53 → random_seed=42 → input_size=52 → batch_size=256 → learning_rate=0.001 → optimizer=<class 'torch.optim.adamw.AdamW'> → activation=ReLU → WMAE=1587.5859
